In [1]:
import sys
import os


import pandas as pd
import plotly.express as px


package_path = os.path.abspath(os.path.join(os.getcwd(), "../"))

if package_path not in sys.path:
    sys.path.append(package_path)

from behave.prophet_evaluator import ProphetEvaluator

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_parquet("data/Monthly-train.parquet")
info = pd.read_csv("data/info.csv")

In [5]:
# Prep ts data
TS_ID = "M54"
meta_ts = info.loc[info.M4id == TS_ID]
startdate_ts = pd.to_datetime(meta_ts.StartingDate.values[0])
ts = df.loc[df.V1 == TS_ID].T.dropna().values[1:]
ts = ts.flatten()
ts = pd.DataFrame({"ds": pd.date_range(startdate_ts.date(), periods=len(ts), freq="M"), "y": ts})

In [6]:
fig = px.line(x=ts.ds, y=ts.y, color_discrete_sequence=["#4561a8"])
fig.update_layout(title="Example Time Series M54", xaxis_title="Date", yaxis_title="Value", width=700, height=400)
fig

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


## Interventions Overview

In [7]:
evaluator = ProphetEvaluator(ts, "y", "ds", "structural", sim_runs=10)
evaluator.evaluate()

 simulation runs: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it]


In [8]:
fig = evaluator.plot_interventions()
fig.update_layout(width=700, height=2900)
fig

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Global Results

In [9]:
(adapt_hist_fig, share_box_fig, mapes_fig) = evaluator.plot_summary() 

The median time to adapt to a shock is 8.5 months.
  Statistic      Value
0      Runs  10.000000
1    Median   8.500000
2      Mean   6.500000
3       Std   4.780167
4       Min   0.000000
5       Max  12.000000
   run  sim_mape  sim_mape_std  benchmark_mape  adaption_time
0    0  0.224510      0.128244        0.270288              4
1    1  0.246153      0.055290        0.199778              8
2    2  0.234357      0.070352        0.294119              0
3    3  0.226525      0.015702        0.055288             12
4    4  0.294859      0.049536        0.219635              0
5    5  0.318356      0.073523        0.268074              9
6    6  0.102426      0.027456        0.301569              0
7    7  0.204318      0.055233        0.115196             11
8    8  0.242840      0.016519        0.061326             12
9    9  0.256308      0.053677        0.199571              9


In [10]:
adapt_hist_fig.update_layout(width=700, height=400)

In [13]:
share_box_fig.update_layout(width=700, height=400)

In [14]:
mapes_fig.update_layout(width=700, height=400)

## Detailed View

In [11]:
fig = evaluator.plot_detail(run = 0)
fig.update_layout(width=700, height=5000)
fig

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## MAPES

In [12]:
fig = evaluator.plot_mapes(run=0)
fig.update_layout(width=700, height=400)
fig 